<a href="https://colab.research.google.com/github/SURESHBEEKHANI/Advanced-LLM-Fine-Tuning/blob/main/llama_3_2_vision_amazon_product.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Fine-Tuning Llama 3.2 Vision
**fine-tune a multimodal model by Meta AI on the Amazon product dataset using the Unsloth framework**

### 1.Install and upgrade UnSloth library for optimized model training

- Use `%%capture` to suppress installation output in Jupyter/Colab environments.
- Install the `unsloth` package from PyPI for initial setup.
- Uninstall the existing `unsloth` package to ensure a clean installation.
- Upgrade to the latest version of `unsloth` directly from the GitHub repository.

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

### 2. Load the model


Load the `unsloth/Llama-3.2-11B-Vision-Instruct` model using FastVisionModel.

Enable 4-bit quantization to reduce memory usage.
Utilize UnSloth's gradient checkpointing for efficient training and inference.


In [ ]:
from unsloth import FastVisionModel  # Use FastLanguageModel for LLMs if needed
import torch

model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Llama-3.2-11B-Vision-Instruct",
    load_in_4bit=True,  # Use 4-bit precision to reduce memory usage; set to False for 16-bit LoRA
    use_gradient_checkpointing="unsloth",  # Use "unsloth" or True for long context support
)


### 3. Add LoRA fine-tuning configuration for Llama 3.2 Vision model

- Apply LoRA (Low-Rank Adaptation) to fine-tune the vision and language components of the model.
- Enable fine-tuning for vision layers, language layers, attention modules, and MLP modules.
- Configure LoRA parameters such as rank (`r`), alpha (`lora_alpha`), and dropout (`lora_dropout`).


In [ ]:
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True,
    finetune_language_layers   = True,
    finetune_attention_modules = True,
    finetune_mlp_modules       = True,
    r                          = 16,
    lora_alpha                 = 16,
    lora_dropout               = 0,
    bias                       = "none",
    random_state               = 3407,
    use_rslora                 = False,
    loftq_config               = None,
)

### 4.  Load a subset of the Amazon Product Descriptions dataset for Vision-Language tasks

- Load the `philschmid/amazon-product-descriptions-vlm` dataset using the Hugging Face `datasets` library.
- Select a subset of the training data (first 500 samples) for faster experimentation and prototyping.


In [ ]:
from datasets import load_dataset

dataset = load_dataset("philschmid/amazon-product-descriptions-vlm",
                       split="train[0:500]")

dataset

In [ ]:
dataset[45]["description"]

### 5.  Convert Amazon product descriptions dataset into conversation format for Vision-Language Models

- Define a system instruction for generating product descriptions based on images.
- Implement a `convert_to_conversation` function to transform dataset samples into a conversation-like structure suitable for Vision-Language Models (VLMs).
- Apply the transformation to the dataset to create a new `converted_dataset`.


In [ ]:
instruction = """
You are an expert Amazon worker who is good at writing product descriptions.
Write the product description accurately by looking at the image.
"""

def convert_to_conversation(sample):
    conversation = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": instruction},
                {"type": "image", "image": sample["image"]},
            ],
        },
        {
            "role": "assistant",
            "content": [{"type": "text", "text": sample["description"]}],
        },
    ]
    return {"messages": conversation}

# Apply the conversion to each sample in the dataset
converted_dataset = [convert_to_conversation(sample) for sample in dataset]

### 6. Enable inference on Llama 3.2 Vision model for generating product descriptions

- Prepare the model for inference using `FastVisionModel.for_inference`.
- Generate a product description by processing an image and instruction using the Vision-Language Model.
- Use a streaming approach to display generated text in real-time.


In [ ]:
FastVisionModel.for_inference(model)  # Enable for inference!

image = dataset[45]["image"]

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": instruction},
        ],
    }
]
input_text = tokenizer.apply_chat_template(
    messages, add_generation_prompt=True
)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt",
).to("cuda")

from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer, skip_prompt=True)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=128,
    use_cache=True,
    temperature=1.5,
    min_p=0.1
)


### 7. Set up SFTTrainer for fine-tuning Llama 3.2 Vision model on product descriptions

- Enable the model for training using `FastVisionModel.for_training`.
- Configure the `SFTTrainer` from the `trl` library for supervised fine-tuning (SFT) with UnSloth optimizations.
- Use `UnslothVisionDataCollator` to handle multi-modal data efficiently during training.


In [ ]:
from unsloth import is_bf16_supported
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig

FastVisionModel.for_training(model)  # Enable for training!

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=UnslothVisionDataCollator(model, tokenizer),  # Must use!
    train_dataset=converted_dataset,
    args=SFTConfig(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=30,
        learning_rate=2e-4,
        fp16=not is_bf16_supported(),
        bf16=is_bf16_supported(),
        logging_steps=5,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",  # For Weights and Biases
        remove_unused_columns=False,
        dataset_text_field="",
        dataset_kwargs={"skip_prepare_dataset": True},
        dataset_num_proc=4,
        max_seq_length=2048,
    ),
)

### 8. Training the model
Start the training process by running the trainer.train() code.

In [ ]:
trainer_stats = trainer.train()

### 11.Push the trained model to Hugging Face Model Hub

In [ ]:
# Export and save the model to your Hugging Face account
model.push_to_hub_merged(
    "SURESHBEEKHANI/llama_3.2_vision_amazon_product_description",
    tokenizer,
    token=""
)